In [1]:
#Import libraries
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [2]:
df_products_clean = pd.read_csv('data.csv')

# Retrieve the exact product urls
pos = 0
for _ in range(len(df_products_clean)):
    df_products_clean.iloc[pos,3] = 'https://www.fairprice.com.sg/search?query=' + df_products_clean.iloc[pos,0]
    df_products_clean.iloc[pos,8] = 'https://www.fairprice.com.sg/search?query=' + df_products_clean.iloc[pos,5]
    df_products_clean.iloc[pos,13] = 'https://www.fairprice.com.sg/search?query=' + df_products_clean.iloc[pos,10]
    df_products_clean.iloc[pos,18] = 'https://www.fairprice.com.sg/search?query=' + df_products_clean.iloc[pos,15]
    df_products_clean.iloc[pos,23] = 'https://www.fairprice.com.sg/search?query=' + df_products_clean.iloc[pos,20]
    pos += 1
    
df_products_clean.head()

C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\407331625.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://www.fairprice.com.sg/search?query=Red Apples' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,3] = 'https://www.fairprice.com.sg/search?query=' + df_products_clean.iloc[pos,0]
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\407331625.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://www.fairprice.com.sg/search?query=Apricots' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,8] = 'https://www.fairprice.com.sg/search?query=' + df_products_clean.iloc[pos,5]
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\407331625.py:8: FutureWarning: Setting an item of incompatible dtype is depr

,Red,redProductName,redProductPrice,redProductLink,redProductImage,Yellow & Orange,yellowProductName,yellowProductPrice,yellowProductLink,yellowProductImage,...,greenProductName,greenProductPrice,greenProductLink,greenProductImage,Blue & Purple,blueProductName,blueProductPrice,blueProductLink,blueProductImage,Recipes
0,Red Apples,NaN,NaN,https://www.fairprice.com.sg/search?query=Red ...,NaN,Apricots,NaN,NaN,https://www.fairprice.com.sg/search?query=Apri...,NaN,...,NaN,NaN,https://www.fairprice.com.sg/search?query=Avoc...,NaN,Blackberries,NaN,NaN,https://www.fairprice.com.sg/search?query=Blac...,NaN,"Total time: 15 mins, Serves: 2\n\nRainbow Oatm..."
1,Blood Oranges,NaN,NaN,https://www.fairprice.com.sg/search?query=Bloo...,NaN,Dates,NaN,NaN,https://www.fairprice.com.sg/search?query=Dates,NaN,...,NaN,NaN,https://www.fairprice.com.sg/search?query=Gree...,NaN,Blueberries,NaN,NaN,https://www.fairprice.com.sg/search?query=Blue...,NaN,"Total time: 5 minutes, Serves: 2\n\nWild blueb..."
2,Cherries,NaN,NaN,https://www.fairprice.com.sg/search?query=Cher...,NaN,Peaches,NaN,NaN,https://www.fairprice.com.sg/search?query=Peaches,NaN,...,NaN,NaN,https://www.fairprice.com.sg/search?query=Gree...,NaN,Black Currants,NaN,NaN,https://www.fairprice.com.sg/search?query=Blac...,NaN,"Total time: 45 minutes, Serves: 6\n\nCherry an..."
3,Cranberries,NaN,NaN,https://www.fairprice.com.sg/search?query=Cran...,NaN,Yellow Figs,NaN,NaN,https://www.fairprice.com.sg/search?query=Yell...,NaN,...,NaN,NaN,https://www.fairprice.com.sg/search?query=Hone...,NaN,Concord Grapes,NaN,NaN,https://www.fairprice.com.sg/search?query=Conc...,NaN,"Total time: 2 hours, Serves 8\n\nHoney and chi..."
4,Red Grapes,NaN,NaN,https://www.fairprice.com.sg/search?query=Red ...,NaN,Grapefruit,NaN,NaN,https://www.fairprice.com.sg/search?query=Grap...,NaN,...,NaN,NaN,https://www.fairprice.com.sg/search?query=Kiwi...,NaN,Dried Plums,NaN,NaN,https://www.fairprice.com.sg/search?query=Drie...,NaN,"Total time: 30 mins, Serves: 6\n\nSingapore Ro..."


In [3]:
# Create a new instance of the Chrome webdriver
driver = webdriver.Chrome()

# Open the product page red
pos = 0

for _ in range(len(df_products_clean)):
    driver.get(df_products_clean.iloc[pos,3])
    source = driver.page_source
    data = bs(source, 'html.parser')
    
    # Check if there are any matching products on the website
    if data.find('div', class_='sc-408392be-0 capxft') is not None:
    
        # Retrieve product name
        df_products_clean.iloc[pos,1] = data.find('div', class_='sc-408392be-0 capxft').text

        # Retrieve product price
        df_products_clean.iloc[pos,2] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text

        # Retrieve product image
        temp = str(data.find('img', class_='sc-aca6d870-0 janHcI'))
        match = re.search(r'src="(.*?)\?', temp)

        if match:
            print(match.group(1))
            df_products_clean.iloc[pos,4] = match.group(1)
        else:
            print("No match found")
            df_products_clean.iloc[pos,4] = ''

    pos += 1

# Open the product page yellow
pos = 0

for _ in range(len(df_products_clean)):
    driver.get(df_products_clean.iloc[pos,8])
    source = driver.page_source
    data = bs(source, 'html.parser')
    
    # Check if there are any matching products on the website
    if data.find('div', class_='sc-408392be-0 capxft') is not None:
    
        # Retrieve product name
        df_products_clean.iloc[pos,6] = data.find('div', class_='sc-408392be-0 capxft').text

        # Retrieve product price
        df_products_clean.iloc[pos,7] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text

        # Retrieve product image
        temp = str(data.find('img', class_='sc-aca6d870-0 janHcI'))
        match = re.search(r'src="(.*?)\?', temp)

        if match:
            print(match.group(1))
            df_products_clean.iloc[pos,9] = match.group(1)
        else:
            print("No match found")
            df_products_clean.iloc[pos,9] = ''

    pos += 1

# Open the product page white
pos = 0

for _ in range(len(df_products_clean)):
    driver.get(df_products_clean.iloc[pos,13])
    source = driver.page_source
    data = bs(source, 'html.parser')
    
    # Check if there are any matching products on the website
    if data.find('div', class_='sc-408392be-0 capxft') is not None:
    
        # Retrieve product name
        df_products_clean.iloc[pos,11] = data.find('div', class_='sc-408392be-0 capxft').text

        # Retrieve product price
        df_products_clean.iloc[pos,12] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text

        # Retrieve product image
        temp = str(data.find('img', class_='sc-aca6d870-0 janHcI'))
        match = re.search(r'src="(.*?)\?', temp)

        if match:
            print(match.group(1))
            df_products_clean.iloc[pos,14] = match.group(1)
        else:
            print("No match found")
            df_products_clean.iloc[pos,14] = ''

    pos += 1

# Open the product page green
pos = 0

for _ in range(len(df_products_clean)):
    driver.get(df_products_clean.iloc[pos,18])
    source = driver.page_source
    data = bs(source, 'html.parser')
    
    # Check if there are any matching products on the website
    if data.find('div', class_='sc-408392be-0 capxft') is not None:
    
        # Retrieve product name
        df_products_clean.iloc[pos,16] = data.find('div', class_='sc-408392be-0 capxft').text

        # Retrieve product price
        df_products_clean.iloc[pos,17] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text

        # Retrieve product image
        temp = str(data.find('img', class_='sc-aca6d870-0 janHcI'))
        match = re.search(r'src="(.*?)\?', temp)

        if match:
            print(match.group(1))
            df_products_clean.iloc[pos,19] = match.group(1)
        else:
            print("No match found")
            df_products_clean.iloc[pos,19] = ''

    pos += 1

# Open the product page blue
pos = 0

for _ in range(len(df_products_clean)):
    driver.get(df_products_clean.iloc[pos,23])
    source = driver.page_source
    data = bs(source, 'html.parser')
    
    # Check if there are any matching products on the website
    if data.find('div', class_='sc-408392be-0 capxft') is not None:
    
        # Retrieve product name
        df_products_clean.iloc[pos,21] = data.find('div', class_='sc-408392be-0 capxft').text

        # Retrieve product price
        df_products_clean.iloc[pos,22] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text

        # Retrieve product image
        temp = str(data.find('img', class_='sc-aca6d870-0 janHcI'))
        match = re.search(r'src="(.*?)\?', temp)

        if match:
            print(match.group(1))
            df_products_clean.iloc[pos,24] = match.group(1)
        else:
            print("No match found")
            df_products_clean.iloc[pos,24] = ''

    pos += 1

C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Italy Red Pop Apple' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,1] = data.find('div', class_='sc-408392be-0 capxft').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$5.95' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,2] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://media.nedigital.sg/fairprice/fpol

https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13228441_XL1_20230309.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13092794_XL1_20221216.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/12197760_XL1.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13151586_XL1_20220908.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13005330_XL1_20230919.jpg
https://media.nedigital.sg/fairprice/90074788_XL1_20240723210059_cf3cff18244e1369060b088478034844.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13228441_XL1_20230309.jpg
https://media.nedigital.sg/fairprice/90174632_XL1_20240724074742_8650ee3624fb9205cbae842d99936e84.jpg
https://media.nedigital.sg/fairprice/90055920_XL1_20240723015302_f4f75f2dccac8e6cf2a213bbdf8fb9fb.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/12164168_XL1_20230510.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/prod

C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'YayaPapaya Apricot' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,6] = data.find('div', class_='sc-408392be-0 capxft').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$7.00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,7] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://media.nedigital.sg/fairprice/90174

https://media.nedigital.sg/fairprice/90174943_XL1_20240724021154_f89de0c585fb1238b1da6235cb637254.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13203263_XL1_20221122.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13024402_XL1_20210824.jpg
https://media.nedigital.sg/fairprice/90074788_XL1_20240723210059_cf3cff18244e1369060b088478034844.jpg
https://media.nedigital.sg/fairprice/90144359_XL1_20240724091022_c4f9005309363e315354e56cfb65a2fa.jpg
https://media.nedigital.sg/fairprice/90177265_XL1_20240724021501_3aa285a75f33d7554a8633c76befc8d8.jpg
https://media.nedigital.sg/fairprice/90161452_XL1_20240715203729_c2434c7df3b4aaf5d940edef3874a2e2.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13166833_XL1_20221122.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13056926_XL1_20230510.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13024402_XL1_20210824.jpg
https://media.nedigital.sg/fai

C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Sumifru Philippines Banana' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,11] = data.find('div', class_='sc-408392be-0 capxft').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$2.45' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,12] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://media.nedigital.sg/fairp

...

C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Yuan Zhen Yuan Australian Avocado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,16] = data.find('div', class_='sc-408392be-0 capxft').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$4.95' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,17] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://media.nedigita

https://media.nedigital.sg/fairprice/90187042_XL1_20240724081339_009be18191f291d7203c7e19a763ebda.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/10792443_XL1.jpg
https://media.nedigital.sg/fairprice/90074797_XL1_20240724091114_325e541db1f9a19322daf67a3cff8c81.jpg
https://media.nedigital.sg/fairprice/90157236_XL1_20240722103630_251d1856456fe67d21296c5be957c31e.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13044104_XL1_20230717.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/11629373_XL1_20230519.jpg
https://media.nedigital.sg/fairprice/90186439_XL1_20240723204613_4ec0549b9ed017439782c12948d19093.jpg
https://media.nedigital.sg/fairprice/90086693_XL1_20240714073143_d32bfad8f878225ae7e83feba5537427.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/10734024_XL1_20230518.jpg
https://media.nedigital.sg/fairprice/90153099_XL1_20240724091150_53f83175bd25b60db5c895e8928eff8f.jpg
https://media.nedigital.sg/fa

C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:136: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Agroaguiar Blackberry' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,21] = data.find('div', class_='sc-408392be-0 capxft').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$5.50' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_products_clean.iloc[pos,22] = data.find('span', class_='sc-aa673588-1 sc-65bf849-1 kdTuLI cXCGWM').text
C:\Users\tommy\AppData\Local\Temp\ipykernel_22952\2785332263.py:147: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://media.nedigital.sg/fairpri

https://media.nedigital.sg/fairprice/90055918_XL1_20240723020230_6926146394d98deb3d6958d1adad8e62.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/10632060_XL1_20201014.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13058031_XL1_20220926.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13237945_XL1_20230911.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13118671_XL1_20210222.jpg
https://media.nedigital.sg/fairprice/90197928_XL1_20240724053653_ed62cf584da97a24f2520b8cc5187a0b.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/11494982_XL1_20240708.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/275885_XL1.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13003761_XL1_20211122.jpg
https://media.nedigital.sg/fairprice/90150776_XL1_20240723233732_50ecf8d8b9af9635e4a8222bfd959a66.jpg
https://media.nedigital.sg/fairprice/90143677_XL1_20240710215

https://media.nedigital.sg/fairprice/90196364_XL1_20240722142430_e7b690c4cf76ed28c482d5087646fe9f.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/90197560_XL1_20240531.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13042233_XL1_20221123.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13236466_XL1_20230919.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/12533584_XL1_20221122.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13167889_XL1_20230521.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13167889_XL1_20230521.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13167889_XL1_20230521.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/12482836_XL1_20230518.jpg
https://media.nedigital.sg/fairprice/90175107_XL1_20240723200819_63d2c377bd206d2d06dd6a7d5f1e677e.jpg
https://media.nedigital.sg/fairprice/fpol/media/images/produ

In [4]:
df_products_clean

,Red,redProductName,redProductPrice,redProductLink,redProductImage,Yellow & Orange,yellowProductName,yellowProductPrice,yellowProductLink,yellowProductImage,...,greenProductName,greenProductPrice,greenProductLink,greenProductImage,Blue & Purple,blueProductName,blueProductPrice,blueProductLink,blueProductImage,Recipes
0,Red Apples,Italy Red Pop Apple,$5.95,https://www.fairprice.com.sg/search?query=Red ...,https://media.nedigital.sg/fairprice/fpol/medi...,Apricots,YayaPapaya Apricot,$7.00,https://www.fairprice.com.sg/search?query=Apri...,https://media.nedigital.sg/fairprice/90174943_...,...,Yuan Zhen Yuan Australian Avocado,$4.95,https://www.fairprice.com.sg/search?query=Avoc...,https://media.nedigital.sg/fairprice/90187042_...,Blackberries,Agroaguiar Blackberry,$5.50,https://www.fairprice.com.sg/search?query=Blac...,https://media.nedigital.sg/fairprice/90055918_...,"Total time: 15 mins, Serves: 2\n\nRainbow Oatm..."
1,Blood Oranges,Bundaberg Sparkling Fruit Bottle Drink - Blood...,$8.88,https://www.fairprice.com.sg/search?query=Bloo...,https://media.nedigital.sg/fairprice/fpol/medi...,Dates,China Winter Date,$3.95,https://www.fairprice.com.sg/search?query=Dates,https://media.nedigital.sg/fairprice/fpol/medi...,...,Fresh Green Grapes,$4.95,https://www.fairprice.com.sg/search?query=Gree...,https://media.nedigital.sg/fairprice/fpol/medi...,Blueberries,Fresh Blueberries,$3.90,https://www.fairprice.com.sg/search?query=Blue...,https://media.nedigital.sg/fairprice/fpol/medi...,"Total time: 5 minutes, Serves: 2\n\nWild blueb..."
2,Cherries,Pasar Cherries,$6.45,https://www.fairprice.com.sg/search?query=Cher...,https://media.nedigital.sg/fairprice/fpol/medi...,Peaches,Donut Peach,$4.95,https://www.fairprice.com.sg/search?query=Peaches,https://media.nedigital.sg/fairprice/fpol/medi...,...,YayaPapaya Apple Green Granny Smith,$3.52,https://www.fairprice.com.sg/search?query=Gree...,https://media.nedigital.sg/fairprice/90074797_...,Black Currants,NaN,NaN,https://www.fairprice.com.sg/search?query=Blac...,NaN,"Total time: 45 minutes, Serves: 6\n\nCherry an..."
3,Cranberries,Nature's Wonders Premium Whole Dried Cranberries,$4.28,https://www.fairprice.com.sg/search?query=Cran...,https://media.nedigital.sg/fairprice/fpol/medi...,Yellow Figs,NaN,NaN,https://www.fairprice.com.sg/search?query=Yell...,NaN,...,Orgo Fresh Sweet Honeydew Melon Jumbo Size (Ho...,$9.98,https://www.fairprice.com.sg/search?query=Hone...,https://media.nedigital.sg/fairprice/90157236_...,Concord Grapes,Smucker's Sugar Free Jam - Concord Grape,$6.70,https://www.fairprice.com.sg/search?query=Conc...,https://media.nedigital.sg/fairprice/fpol/medi...,"Total time: 2 hours, Serves 8\n\nHoney and chi..."
4,Red Grapes,Egypt Seedless Red Grapes,$4.95,https://www.fairprice.com.sg/search?query=Red ...,https://media.nedigital.sg/fairprice/fpol/medi...,Grapefruit,YayaPapaya Grapefruit,$1.30,https://www.fairprice.com.sg/search?query=Grap...,https://media.nedigital.sg/fairprice/90074788_...,...,Zespri New Zealand Kiwifruit - SunGold,$6.55,https://www.fairprice.com.sg/search?query=Kiwi...,https://media.nedigital.sg/fairprice/fpol/medi...,Dried Plums,Sun Q Organic Soft Dried Plums,$9.19,https://www.fairprice.com.sg/search?query=Drie...,https://media.nedigital.sg/fairprice/fpol/medi...,"Total time: 30 mins, Serves: 6\n\nSingapore Ro..."
5,Grapefruit,YayaPapaya Grapefruit,$1.30,https://www.fairprice.com.sg/search?query=Grap...,https://media.nedigital.sg/fairprice/90074788_...,Golden Kiwifruit,NaN,NaN,https://www.fairprice.com.sg/search?query=Gold...,NaN,...,Simply Finest Small Lime,$1.65,https://www.fairprice.com.sg/search?query=Limes,https://media.nedigital.sg/fairprice/fpol/medi...,Elderberries,Sambucol Black Elderberry - Kids,$29.44,https://www.fairprice.com.sg/search?query=Elde...,https://media.nedigital.sg/fairprice/fpol/medi...,"Total time: 1 hour, Serves: 4\n\nRoasted curri..."
6,Red Pears,Italy Red Pop Apple,$5.95,https://www.fairprice.com.sg/search?query=Red ...,https://media.nedigital.

In [5]:
# Export to csv file
df_products_clean.to_csv('products.csv')